## Fish Disease Knowledge Graph Creation

This notebook reads a csv file which is a dataset on fish diseases and symptoms associated, and creates a neo4j knowledge graph database using the relationship between the fish diseases and symptoms.# Install below Python-NEO4J interaction library
!pip install neo4j

In [3]:
# Install below Python-NEO4J interaction library
#!pip install neo4j

### Import library

In [4]:
## import library
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import logging
import pandas as pd

ModuleNotFoundError: No module named 'neo4j'

### Connect to neo4j 

In [ ]:
# Initialize the graph DB 
graph = GraphDatabase.driver(
    "neo4j://localhost:7687",
    auth=("neo4j", "Dragonknight94!")
)

In [ ]:
# Delete all the nodes and relationships
query = (
        "MATCH (all_nodes)"
        "OPTIONAL MATCH (all_nodes)-[all_rels]->()"
        "DELETE all_nodes, all_rels"
    )
with graph.session() as session:
    result = session.run(query)

### Load Fish Disease dataset

In [ ]:
filename = 'FishDiseases(clean) - Disease.csv'
folderpath = r'/home/jengwei/nus/1. Intelligent Reasoning Systems/1. Machine Reasoning/MVP Project/Dataset//'

disease_df = pd.read_csv(folderpath + filename, encoding = "ISO-8859-1", index_col='ID')
disease_df

,Disease1,Disease2,Native_Water,Fish,Symptom1,Symptom2,Symptom3,Symptom4,Symptom5,Symptom6,...,Treatments,Vet_Advised,Medicine1,Medicine2,Medicine3,Medicine4,Medicine5,Medicine6,Medicine7,Medicine8
ID,,,,,,,,,,,,,,,,,,,,,
0,Fin or tail rot,Aeromonas salmonicida,ALL,NaN,Fins or tail edges appear white,Frayed and ragged fins,Frayed Tail,Swimming difficulty,Loss of appetite,Settle Bottom of Tank,...,Several antibiotics are effective in treating ...,True,Antibiotics for Gram-Negative Organisms,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cloudy Eye,NaN,ALL,NaN,Cloudy white or greyish eyes,Loss of vision,Show signs of distress,NaN,NaN,NaN,...,Change the filter (or clean it) and get rid of...,True,Strong Antibiotics,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Columnaris,Cotton Mouth,Freshwater,NaN,Frayed and ragged fins,Skin Ulcer,Skin Peeling,Greyish or White or cloudy patches on the gill,"Mucus on the gills, head, and dorsal regions",Light or dark brown gills,...,External infections should be treated with ant...,False,Copper Sulfate,Acriflavine,Furan,Terramycin,Salt,NaN,NaN,NaN
3,Dropsy,NaN,ALL,NaN,Bloated abdomen or belly,Scales stick out,Eyes protruding,Lethargic behavior,Loss of appetite,Rapid gill movement,...,The infection-causing dropsy is not easily cur...,False,Salt,Antibiotics for Gram-Negative Bacteria,NaN,NaN,NaN,NaN,NaN,NaN
4,Fish TuberculosisÂ,Mycobacterium spp,ALL,NaN,Lethargic behavior,Hollowed belly,Turn pale,Skin Ulcer,Frayed and ragged fins,Fin and scale loss,...,There is no effective treatment of fish tuberc...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Viral Hemorrhagic SepticemiaÂ,Egtved virus,ALL,NaN,Bright red streaks on fins,Patchy or darker body,Eyes protruding,Bloated abdomen or belly,Signs of bleeding,Breathing rapidly,...,"As with many viral diseases of fish, there is ...",True,Levamisole,Metronidazole,Praziquantel,Nitrofurazone (Antibiotics),Erythromycin (Antibiotics),NaN,NaN,NaN
6,Freshwater Ich,White Spot,Freshwater,NaN,White spots on the body and gills,Clamped fins,Lethargic behavior,Breathing rapidly,Loss of appetite,Flashing or Rub against bottom or objects in t...,...,Have a veterinarian examine your sick fish to ...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Popeye,Exophthalmia,ALL,NaN,Eyes protruding,Cloudy white or greyish eyes,Bloodstained eye,Ruptured eye,Loss of vision,Lethargic behavior,...,Treatment for popeye will depend upon the unde...,True,Salt,Antibiotics,NaN,NaN,NaN,NaN,NaN,NaN
8,Swim Bladder Disease,Swim Bladder Disorder,ALL,NaN,Bloated abdomen or belly,Curved spine,Swimming difficulty,Upside down,Erratic swimming,Swimming with heavy head,...,"Treatment involves water maintenance, feeding ...",False,Pea,Salt,Stress Coat Water Conditioner,NaN,NaN,NaN,NaN,NaN


In [ ]:
disease_df = disease_df.fillna(' ')
disease_df.head()

,Disease1,Disease2,Native_Water,Fish,Symptom1,Symptom2,Symptom3,Symptom4,Symptom5,Symptom6,...,Treatments,Vet_Advised,Medicine1,Medicine2,Medicine3,Medicine4,Medicine5,Medicine6,Medicine7,Medicine8
ID,,,,,,,,,,,,,,,,,,,,,
0,Fin or tail rot,Aeromonas salmonicida,ALL,,Fins or tail edges appear white,Frayed and ragged fins,Frayed Tail,Swimming difficulty,Loss of appetite,Settle Bottom of Tank,...,Several antibiotics are effective in treating ...,True,Antibiotics for Gram-Negative Organisms,,,,,,,
1,Cloudy Eye,,ALL,,Cloudy white or greyish eyes,Loss of vision,Show signs of distress,,,,...,Change the filter (or clean it) and get rid of...,True,Strong Antibiotics,,,,,,,
2,Columnaris,Cotton Mouth,Freshwater,,Frayed and ragged fins,Skin Ulcer,Skin Peeling,Greyish or White or cloudy patches on the gill,"Mucus on the gills, head, and dorsal regions",Light or dark brown gills,...,External infections should be treated with ant...,False,Copper Sulfate,Acriflavine,Furan,Terramycin,Salt,,,
3,Dropsy,,ALL,,Bloated abdomen or belly,Scales stick out,Eyes protruding,Lethargic behavior,Loss of appetite,Rapid gill movement,...,The infection-causing dropsy is not easily cur...,False,Salt,Antibiotics for Gram-Negative Bacteria,,,,,,
4,Fish TuberculosisÂ,Mycobacterium spp,ALL,,Lethargic behavior,Hollowed belly,Turn pale,Skin Ulcer,Frayed and ragged fins,Fin and scale loss,...,There is no effective treatment of fish tuberc...,False,,,,,,,,


### Create Nodes and Relationship links in neo4j

In [ ]:
for i in range(disease_df.shape[0]):
    ## traverse row-by-row
    # add nodes for disease
    query = (
        "MERGE (node: "+"Disease"+" {name: $name, native_water: $native_water, treatments: $treatments, vet_advised: $vet_advised, aka: $aka})"
        "RETURN node"
    )
    with graph.session() as session:
        result = session.run(query, name=disease_df.loc[i, 'Disease1'], native_water=disease_df.loc[i, 'Native_Water'], treatments=disease_df.loc[i, 'Treatments'], vet_advised=disease_df.loc[i, 'Vet_Advised']\
                             , aka=disease_df.loc[i, 'Disease2'])
    print("create new node with label as {0} and name as {1}".format("Disease", disease_df.loc[i, 'Disease1']))
            
    # add symptom nodes and relationship to disease
    for j in range(4, 16):
        # iterate across the associated symptoms  
        if disease_df.iloc[i,j] != ' ':
            # add symptom node
            query = (
                "MERGE (node: "+"Symptom"+" {name: $name})"
                "RETURN node"
            )
            with graph.session() as session:
                result = session.run(query, name=disease_df.iloc[i,j])
            print("create new node with label as {0} and name as {1}".format("Symptom", disease_df.iloc[i,j]))
            
            # add relationship
            query = (
                "MATCH (n1 {name: $name1})"
                "MATCH (n2 {name: $name2})"
                "MERGE (n1) - [r:"+"symptom_of"+"] -> (n2)"   
                "RETURN n1, n2, r"
            )

            with graph.session() as session:
                result = session.run(query, name1=disease_df.iloc[i,j], name2=disease_df.iloc[i,0])
            print("create new relationship {0} - symptom_of -> {1}".format(disease_df.iloc[i,j], disease_df.iloc[i,0]))


create new node with label as Disease and name as Fin or tail rot
create new node with label as Symptom and name as Fins or tail edges appear white
create new relationship Fins or tail edges appear white - symptom_of -> Fin or tail rot
create new node with label as Symptom and name as Frayed and ragged fins
create new relationship Frayed and ragged fins - symptom_of -> Fin or tail rot
create new node with label as Symptom and name as Frayed Tail
create new relationship Frayed Tail - symptom_of -> Fin or tail rot
create new node with label as Symptom and name as Swimming difficulty
create new relationship Swimming difficulty - symptom_of -> Fin or tail rot
create new node with label as Symptom and name as Loss of appetite
create new relationship Loss of appetite - symptom_of -> Fin or tail rot
create new node with label as Symptom and name as Settle Bottom of Tank
create new relationship Settle Bottom of Tank - symptom_of -> Fin or tail rot
create new node with label as Disease and name

### Load Symptoms dataset

In [ ]:
filename = 'FishDiseases(clean) - Symptom.csv'
folderpath = r'/home/jengwei/nus/1. Intelligent Reasoning Systems/1. Machine Reasoning/MVP Project/Dataset//'

symptom_df = pd.read_csv(folderpath + filename, encoding = "ISO-8859-1")
symptom_df

,Symptom,SymptomType,SymptomCategory1,SymptomCategory2,Question,ImageURL,Disease_List
0,Bloated abdomen or belly,Physical,Body,NaN,NaN,NaN,NaN
1,Bloodstained eye,Physical,Eye,NaN,NaN,NaN,NaN
2,Breathing rapidly,Behaviourial,Breathing,NaN,NaN,NaN,NaN
3,Bright red streaks on fins,Physical,Fin,NaN,NaN,NaN,NaN
4,Clamped fins,Physical,Fin,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
67,White stringy faeces,Physical,Body,NaN,NaN,NaN,NaN
68,Whte or pinkish bumps on skin or fins,Physical,Skin,Fin,NaN,NaN,NaN
69,Worm on body,Physical,Body,NaN,NaN,NaN,NaN
70,Worm seen at anus,Physical,Body,NaN,NaN,NaN,NaN


In [ ]:
# fill nan values with empty string
symptom_df = symptom_df.fillna(' ')
symptom_df.head()

,Symptom,SymptomType,SymptomCategory1,SymptomCategory2,Question,ImageURL,Disease_List
0,Bloated abdomen or belly,Physical,Body,,,,
1,Bloodstained eye,Physical,Eye,,,,
2,Breathing rapidly,Behaviourial,Breathing,,,,
3,Bright red streaks on fins,Physical,Fin,,,,
4,Clamped fins,Physical,Fin,,,,


In [ ]:
# Iterate over all the symptoms and set properties
for i in range(symptom_df.shape[0]):
    
    symptom_type = symptom_df.loc[i, 'SymptomType']
    category1 = symptom_df.loc[i, 'SymptomCategory1']
    category2 = symptom_df.loc[i, 'SymptomCategory2']
    question = symptom_df.loc[i, 'Question']
    imageUrl = symptom_df.loc[i, 'ImageURL']
    
    query = (
        "MATCH (n:Symptom{name: $name})"
        #"MERGE (n1) - [r:"+"symptom_of"+"] -> (n2)"
        "SET n.type = '"+symptom_type+"'"
        "SET n.category1 = '"+category1+"'"
        "SET n.category2 = '"+category2+"'"
        "SET n.question = '"+question+"'"
        "SET n.imageURL = '"+imageUrl+"'"
        "RETURN n;"
    )
    
    with graph.session() as session:
        result = session.run(query, name=symptom_df.loc[i,'Symptom'])
    print("Updated node {0}".format(symptom_df.loc[i,'Symptom']))

Updated node Bloated abdomen or belly
Updated node Bloodstained eye
Updated node Breathing rapidly
Updated node Bright red streaks on fins
Updated node Clamped fins
Updated node Cloudy Skin
Updated node Cloudy white or greyish eyes
Updated node Cotton like patchs on scale
Updated node Cotton wool like growth
Updated node Curved spine
Updated node Dots on fish body or condensed at fin attachments
Updated node Erratic swimming
Updated node Eyes protruding
Updated node Fin and scale loss
Updated node Fine yellow or light brown rust or film on skin
Updated node Fins or tail edges appear white
Updated node Flashing or Rub against bottom or objects in the tank
Updated node Frayed and ragged fins
Updated node Frayed Tail
Updated node Gasping at the surface
Updated node Gills not fully close
Updated node grey or white blue mucus
Updated node Greyish or cream patches on skin and fins
Updated node Greyish or White or cloudy patches on the gill
Updated node Greyish or White or cloudy patches on t

In [ ]:
## Add imageurl to Symptom attribute

### Visualize created knowledge graph at: http://localhost:7474/browser/

* Username: neo4j

* Password: ai-user